In [318]:
import random
#random.seed(42)

In [319]:
!pip install pyppeteer

In [320]:
import asyncio
import re
import csv
import pyppeteer as ptr
import pandas as pd
import nltk
from typing import Optional, TypeVar
from typing import Callable
from glob import glob
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [321]:
import warnings
warnings.filterwarnings('ignore')

In [322]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [323]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [324]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [325]:
resource_path = "/content/drive/MyDrive/resources/"

data_path = resource_path + "data/"
data_source_path = data_path + "sources"
data_target_path = data_path + "targets"

In [326]:
from typing import List, Tuple
PosTag = Tuple[str, str]
PosTagList = List[PosTag]
StemWord = str
StemWordList = List[StemWord]

In [327]:
def get_name_and_extension(file_path: str) -> Tuple[str, str]:
    regex = re.compile(r"(.*)/(.*)\.(.*)")
    return regex.match(file_path).group(2,3)

In [328]:
!pip install nest_asyncio
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (95.0.4638.69-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [329]:
!dpkg -L chromium-chromedriver

/.
/usr
/usr/bin
/usr/bin/chromedriver
/usr/lib
/usr/lib/chromium-browser
/usr/share
/usr/share/doc
/usr/share/doc/chromium-chromedriver
/usr/share/doc/chromium-chromedriver/copyright
/usr/lib/chromium-browser/chromedriver
/usr/share/doc/chromium-chromedriver/changelog.Debian.gz


In [330]:
## Scraper.
async def download_html(browser: ptr.browser.Browser, page: ptr.page.Page, url: str, selector: str) -> Optional[str]:
    await page.goto(url, waitUntil="load", timeout=0)
    content = await page.querySelector(selector)

    html = ''
    if content:
        html = await page.evaluate('(element) => element.textContent', content)
        
    return html

def write_to_resource_target(target_path: str, file_name: str, content: StemWordList, extension="txt") -> None:
    with open(f"{target_path}/{file_name}.{extension}", "w") as file:
        file.write("\n".join(content))


async def get_training_data_from_folder(source_path: str, target_path: str, force=False) -> None:
    browser = await ptr.launch(headless=True, executable_path="/usr/lib/chromium-browser/chromedriver", options={'args': ['--no-sandbox']})
    page = await browser.newPage()
    
    glob_pattern = "/**/*"
    source_files = glob(source_path + glob_pattern, recursive=True)
    target_files = glob(target_path + glob_pattern, recursive=True)
    target_file_names = [get_name_and_extension(file_path)[0] for file_path in target_files]
    
    for file_path in source_files:
        [file_name, file_extension] = get_name_and_extension(file_path)
        
        if not file_name in target_file_names or force:
            result = await get_training_data(browser, page, file_path)
            write_to_resource_target(target_path, file_name, result)        
                
    await browser.close()

async def get_training_data(browser: ptr.browser.Browser, page: ptr.page.Page, file_path: str) -> StemWordList:
    df = pd.read_csv(file_path)
    df.rename(columns=lambda x: x.strip(), inplace=True)

    words = []
    if isinstance(df, pd.DataFrame):
        for _, row in df.iterrows():
            result = await download_html(browser, page, row["link"], row["selector"])
            words.extend(clean_up_words(tokenize(result)))
    
    return words

In [331]:
## Cleaners.
T = TypeVar("T")
R = TypeVar("R")

TestValueStrCallable = Callable[[T], str]
TestValueBoolCallable = Callable[[T], bool]
ValueTestFnCallable = Callable[[T], TestValueStrCallable]
FilterCallable = Callable[[ValueTestFnCallable], bool]
MapCallable = Callable[[ValueTestFnCallable], str]

def tokenize(data: str) -> PosTagList:
    tokenized_words = nltk.word_tokenize(data)
    mutated_words = nltk.pos_tag(tokenized_words)
    return mutated_words

def filter_words(x: T, test_value: TestValueStrCallable, *fns: FilterCallable) -> bool:
    if fns:
        if fns[0](x, test_value):
            return filter_words(x, test_value, *fns[1:])
        else:
            return False
    
    return True
    
def filter_by_punctuation(x: T, test_value: TestValueStrCallable = lambda t: t[0]) -> bool:
    return x[0] != x[1]

def filter_by_stop_word(x: T, test_value: TestValueStrCallable = lambda t: t[0]) -> bool:
    return test_value(x) not in stopwords.words("english")

def regex_filter(regex: str):
    def filter_by_regex(x: T, test_value: TestValueStrCallable = lambda t: t[0]):
        rgx = re.compile(regex)
        return rgx.match(test_value(x))
    return filter_by_regex

filter_by_alphabet = regex_filter(r"^([a-zA-Z]|')+$")
filter_by_apostrophe = regex_filter(r"^[^']*$")

def map_by_stem_words(x: PosTag, test_value: TestValueStrCallable = lambda t: t[0], ps=PorterStemmer()) -> StemWord:
    return ps.stem(test_value(x)).lower()

def map_words(x: T, test_value: TestValueStrCallable, *fns: MapCallable) -> StemWord:
    if fns:
        return map_words(fns[0](test_value(x)), test_value, *fns[1:])

    return x
            
def clean_up_words(words: PosTagList) -> StemWordList:
    filtered_words = list(
        filter(
        lambda x: filter_words(x, lambda x: x[0], filter_by_punctuation, filter_by_stop_word, filter_by_alphabet),
        words
        )
    )

    stem_words = list(map(lambda x: map_words(x, lambda x: x, map_by_stem_words), filtered_words))
    
    return list(
        filter(
            lambda x: filter_words(x, lambda x: x, filter_by_apostrophe), 
            stem_words)
    )

In [332]:
import nest_asyncio
nest_asyncio.apply()

In [333]:
# type: ignore
#asyncio.run(get_training_data_from_folder(resource_source_path, resource_target_path))

In [334]:
# Parsers
from typing import Dict

def get_cleaned_data_from_file(input_file_path: str) -> Optional[List[str]]:
    with open(input_file_path, "r") as file:
        return [word.strip("\n") for word in list(file.readlines())]

def get_cleaned_data_from_folder(input_path: str) -> Dict[str, PosTagList]:
    glob_pattern = "/**/*"
    input_files = glob(input_path + glob_pattern, recursive=True)
    
    data = dict()
    for file_path in input_files:
        [file_name, file_extension] = get_name_and_extension(file_path)
        classification_data = get_cleaned_data_from_file(file_path)
        data[file_name] = classification_data
        
    return data


def group_by_tags(pos_tag_list: PosTagList) -> Dict[str, List[str]]:
    """
      @unused
    """
    groups = dict()
    
    for value,tag in pos_tag_list:
        if tag in groups:
            groups[tag]
            groups[tag].append(value)
        else:
            groups[tag] = [value]
            
    return groups

In [335]:
from nltk.probability import WittenBellProbDist, FreqDist,LaplaceProbDist
import numpy as np

def unpack_dict_list(dict_list: Dict[str, StemWordList]):
    return [v for k in dict_list for v in dict_list[k]]

def generate_sample(population: StemWordList, label: str, n_unique_words: int, word_limit = 56):
    freq_dist = FreqDist(population)
    prob_dist = WittenBellProbDist(freq_dist, n_unique_words)
    
    new_word_limit = word_limit #random.randint(1, word_limit)

    samples = [prob_dist.generate() for _ in range(word_limit)]

    return np.array(list([*samples, label])).reshape(-1, 1)

def generate_samples(data: Dict[str, StemWordList], n_samples = 1000):
    n_unique_words = len(set(unpack_dict_list(data)))
    
    return np.array([np.array([generate_sample(data[k], k, n_unique_words) for _ in range(n_samples)]) for k in data])

In [336]:
from sklearn.preprocessing import OneHotEncoder,LabelBinarizer
from typing import Union

def get_one_hot_encoder(population: Dict[str, StemWordList]):
    population_array = np.array(list(set(unpack_dict_list(population)))).reshape(-1, 1)
    encoder = OneHotEncoder(handle_unknown='ignore')
    encoder.fit(population_array)
    return encoder

def get_label_encoder(labels: np.ndarray):
    encoder = LabelBinarizer()
    encoder.fit(labels)
    return encoder

def transform_data_by_encoder(data: np.ndarray, encoder: Union[OneHotEncoder, LabelBinarizer]):
    encoded_data = encoder.transform(data)
    
    if not isinstance(encoded_data, np.ndarray):
        encoded_data = encoded_data.toarray()
        
    return encoded_data

def split_into_x_y(samples: np.ndarray):
    x = samples[:,:,:-1]
    y = samples[:,:,-1]
    y = y.reshape(y.shape[0]*y.shape[1], 1)
    return (x,y)
    
def transform_x_y(x: np.ndarray, y: np.ndarray, in_coder: OneHotEncoder, out_coder: LabelBinarizer):   
    encoded_x = np.stack([
        transform_data_by_encoder(x[lbl_idx, smpl_idx], in_coder)
        for lbl_idx in range(x.shape[0])
        for smpl_idx in range(x.shape[1])
    ], axis=0)
            
    encoded_y = transform_data_by_encoder(y, out_coder)
    
    return (encoded_x, encoded_y)


In [337]:
import pickle

model_path = resource_path + "models/"
input_encoder_file_name = "input_encoder.pickle"
output_encoder_file_name = "output_encoder.pickle"

def get_x_y(path = data_target_path, n_samples=200):
    result = get_cleaned_data_from_folder(data_target_path)
    samples = generate_samples(result, n_samples)
    x,y = split_into_x_y(samples)

    encoder_paths = glob(model_path + "*.pickle", recursive=True)
    encoder_file_names = [get_name_and_extension(file_path)[0] for file_path in encoder_paths]

    if input_encoder_file_name in encoder_file_names:
      input_encoder = pickle.load(open(encoder_paths[encoder_file_names.index(input_encoder_file_name)], "rb"))
    else:
      input_encoder = get_one_hot_encoder(result)

    if output_encoder_file_name in encoder_file_names:
      output_encoder = pickle.load(open(encoder_paths[encoder_file_names.index(output_encoder_file_name)], "rb"))
    else:
      output_encoder = get_label_encoder(y)

    # DUMP
    pickle.dump(input_encoder, open(model_path + input_encoder_file_name, "wb"))
    pickle.dump(output_encoder, open(model_path + output_encoder_file_name, "wb"))

    return transform_x_y(x, y, input_encoder, output_encoder)



In [338]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def clean_up_user_input(sentence, word_limit=56):
  tokenized_words = tokenize(sentence)
  word_to_test = np.array(clean_up_words(tokenized_words)).reshape(-1, 1)

  input_encoder = pickle.load(open(model_path + input_encoder_file_name, "rb"))

  encoded_data = transform_data_by_encoder(word_to_test, input_encoder)
  difference = word_limit - encoded_data.shape[0]
  zero_array = np.zeros(encoded_data.shape[1])

  for _ in range(difference):
    encoded_data = np.append(encoded_data, [zero_array], axis=0)

  return encoded_data

In [339]:
def get_data(n_samples=200, test_size=0.4, random_state=random.randint(0, 100)):
    x, y = get_x_y(n_samples=n_samples)
    x=x.reshape((*x.shape, 1))
    return train_test_split(x, y, test_size=test_size, random_state=random_state)

x_train, x_test, y_train, y_test = get_data()

In [347]:
from keras import regularizers, activations

def run():
    input_shape = x_train.shape[1:]
    vocab_size = input_shape[-2]

    config = {}

    config["dense_layer_5"] = {
        "units": 16,
        "activation": "relu",
        "bias_regularizer": regularizers.l1_l2(0.2, 0.2)
    }

    config["dense_layer_5"] = {
        "units": 8,
        "activation": "relu",
        "bias_regularizer": regularizers.l1_l2(0.4, 0.2)
    }

    # config["conv_layer_0"] = {
    #     "filters": 1,
    #     "kernel_size": (2, vocab_size),
    #     "strides": (1,1),
    #     "activation": "relu",
    # }

    # config["pool_layer_0"] = {
    #     "pool_size": (3,3),
    #     "padding": "same"
    # }

    # config["conv_layer_1"] = {
    #     "filters": 1,
    #     "kernel_size": (2, 1),
    #     "strides": (1,1),
    #     "activation": "relu"
    # }

    # config["pool_layer_1"] = {
    #     "pool_size": (2,2),
    #     "padding": "same",
    # }

    config["flatten_layer_0"] = {
    }

    config["dense_layer_0"] = {
        "units": 4,
        "activation": "relu",
        # "kernel_regularizer": regularizers.l1_l2(0.1, 0.2),
        # "bias_regularizer": regularizers.l1_l2(0.1, 0.2)
    }

    config["dropout_layer_0"] = {
        "rate": 0.5,
    }

    config["dense_layer_1"]= {
        "units": 8,
        "activation": "relu",
        # "kernel_regularizer": regularizers.l1_l2(0.1, 0.2),
        # "bias_regularizer": regularizers.l1_l2(0.1, 0.2)
    }

    config["dropout_layer_1"]= {
        "rate": 0.5
    }

    config["dense_layer_final"] = {
        "units": (y_train.shape[-1]),
        "activation": "softmax"
    }

    config["comp_model"]= {
        "optimizer": tf.keras.optimizers.Adam(0.001),
        "loss": keras.losses.binary_crossentropy,
        "metrics": ["accuracy", keras.metrics.Precision()]
    }

    encoder_input = keras.Input(shape=input_shape)
    prev_layer = encoder_input
    encoder_output=None

    layer_types = {
        "dense": layers.Dense,
        "flatten": layers.Flatten,
        "dropout": layers.Dropout,
        "conv": layers.Conv2D,
        "pool": layers.MaxPool2D,
        "comp": None,
    }
    
    for k,v in config.items():
      type_of_layer, *_ = k.split("_")
      if type_of_layer != "comp":
        print(v)
        encoder_output = layer_types[type_of_layer](**v)(prev_layer)
        prev_layer = encoder_output
    
    model = keras.Model(encoder_input, encoder_output, name="encoder")

    print(model.summary())
    
    model.compile(**config["comp_model"])

    history = model.fit(x_train, y_train, batch_size=2, epochs=16, validation_split=0.2)

    test_scores = model.evaluate(x_test, y_test, verbose=2)

    print(y_train[0])

    print("Test loss:", test_scores[0])
    print("Test accuracy:", test_scores[1])

    return model    
    
model = run()

{'units': 8, 'activation': 'relu', 'bias_regularizer': <keras.regularizers.L1L2 object at 0x7f9baced9450>}
{}
{'units': 4, 'activation': 'relu'}
{'rate': 0.5}
{'units': 8, 'activation': 'relu'}
{'rate': 0.5}
{'units': 3, 'activation': 'softmax'}
Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_68 (InputLayer)       [(None, 56, 4461, 1)]     0         
                                                                 
 dense_156 (Dense)           (None, 56, 4461, 8)       16        
                                                                 
 flatten_53 (Flatten)        (None, 1998528)           0         
                                                                 
 dense_157 (Dense)           (None, 4)                 7994116   
                                                                 
 dropout_90 (Dropout)        (None, 4)                 0         
           

In [355]:
result = unpack_dict_list(get_cleaned_data_from_folder(data_target_path))

for i in result:
  print(i)

  word = clean_up_user_input(i)
  word = word.reshape(*word.shape, 1)
  words = np.array([word])
  output_encoder: LabelBinarizer = pickle.load(open(model_path + output_encoder_file_name, "rb"))
  prediction = model.predict(words)
  print(prediction)
  prediction_binary = np.zeros_like(prediction)
  prediction_binary[:,prediction.argmax(1)] = 1
  print(prediction_binary)
  print(prediction)
  print(output_encoder.inverse_transform(prediction))




introduct
[[0.36861306 0.35012847 0.28125846]]
[[1. 0. 0.]]
[[0.36861306 0.35012847 0.28125846]]
['depression']
migrain
[[0.36861306 0.35012847 0.28125846]]
[[1. 0. 0.]]
[[0.36861306 0.35012847 0.28125846]]
['depression']
common
[[0.36861306 0.35012847 0.28125846]]
[[1. 0. 0.]]
[[0.36861306 0.35012847 0.28125846]]
['depression']
episod
[[0.36861306 0.35012847 0.28125846]]
[[1. 0. 0.]]
[[0.36861306 0.35012847 0.28125846]]
['depression']
disord
[[0.36861306 0.35012847 0.28125846]]
[[1. 0. 0.]]
[[0.36861306 0.35012847 0.28125846]]
['depression']
hallmark
[[0.36861306 0.35012847 0.28125846]]
[[1. 0. 0.]]
[[0.36861306 0.35012847 0.28125846]]
['depression']
disabl
[[0.36861306 0.35012847 0.28125846]]
[[1. 0. 0.]]
[[0.36861306 0.35012847 0.28125846]]
['depression']
headach
[[0.36861306 0.35012847 0.28125846]]
[[1. 0. 0.]]
[[0.36861306 0.35012847 0.28125846]]
['depression']
gener
[[0.36861306 0.35012847 0.28125846]]
[[1. 0. 0.]]
[[0.36861306 0.35012847 0.28125846]]
['depression']
associ
[[0.36

ValueError: ignored